In [6]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=015b78bc0e9e010355b70d3100f0e8c4d295dc6e4a39ff1743c4b1c4087ca8b7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [61]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import lit
from pyspark.ml.feature import Imputer

In [5]:
spark = SparkSession.builder.appName("pyspark").getOrCreate()
spark

In [19]:
#ways to read csv file using spark session
df = spark.read.csv("/content/drive/MyDrive/dataset/Zoo.csv")
df = spark.read.csv("/content/drive/MyDrive/dataset/Zoo.csv", header=True)
df = spark.read.options(header=True, inferSchema=True).csv('/content/drive/MyDrive/dataset/Zoo.csv')
df.show()

+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+
|animal name|hair|feathers|eggs|milk|airborne|aquatic|predator|toothed|backbone|breathes|venomous|fins|legs|tail|domestic|catsize|type|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+
|   aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|
|   antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|   1|
|       bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|   4|
|       bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|
|       boar|   1|       0|   0|   1|       0|  

In [28]:
type(df)
df.dtypes

[('animal name', 'string'),
 ('hair', 'int'),
 ('feathers', 'int'),
 ('eggs', 'int'),
 ('milk', 'int'),
 ('airborne', 'int'),
 ('aquatic', 'int'),
 ('predator', 'int'),
 ('toothed', 'int'),
 ('backbone', 'int'),
 ('breathes', 'int'),
 ('venomous', 'int'),
 ('fins', 'int'),
 ('legs', 'int'),
 ('tail', 'int'),
 ('domestic', 'int'),
 ('catsize', 'int'),
 ('type', 'int')]

In [30]:
df.printSchema()
df.describe()

root
 |-- animal name: string (nullable = true)
 |-- hair: integer (nullable = true)
 |-- feathers: integer (nullable = true)
 |-- eggs: integer (nullable = true)
 |-- milk: integer (nullable = true)
 |-- airborne: integer (nullable = true)
 |-- aquatic: integer (nullable = true)
 |-- predator: integer (nullable = true)
 |-- toothed: integer (nullable = true)
 |-- backbone: integer (nullable = true)
 |-- breathes: integer (nullable = true)
 |-- venomous: integer (nullable = true)
 |-- fins: integer (nullable = true)
 |-- legs: integer (nullable = true)
 |-- tail: integer (nullable = true)
 |-- domestic: integer (nullable = true)
 |-- catsize: integer (nullable = true)
 |-- type: integer (nullable = true)



DataFrame[summary: string, animal name: string, hair: string, feathers: string, eggs: string, milk: string, airborne: string, aquatic: string, predator: string, toothed: string, backbone: string, breathes: string, venomous: string, fins: string, legs: string, tail: string, domestic: string, catsize: string, type: string]

In [26]:
# selecting rows and columns
df.columns
df.head(5)

#selecting single col with values
df.select("hair").show()
type(df.select("hair"))

#selecting multiple col with values
df.select(["hair", "legs"]).show()

+----+
|hair|
+----+
|   1|
|   1|
|   0|
|   1|
|   1|
|   1|
|   1|
|   0|
|   0|
|   1|
|   1|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   1|
|   0|
|   0|
+----+
only showing top 20 rows

+----+----+
|hair|legs|
+----+----+
|   1|   4|
|   1|   4|
|   0|   0|
|   1|   4|
|   1|   4|
|   1|   4|
|   1|   4|
|   0|   0|
|   0|   0|
|   1|   4|
|   1|   4|
|   0|   2|
|   0|   0|
|   0|   0|
|   0|   4|
|   0|   6|
|   0|   2|
|   1|   4|
|   0|   0|
|   0|   0|
+----+----+
only showing top 20 rows



In [31]:
df = df.withColumnRenamed("legs", "no_of_legs")
df.show()

+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----------+----+--------+-------+----+
|animal name|hair|feathers|eggs|milk|airborne|aquatic|predator|toothed|backbone|breathes|venomous|fins|no_of_legs|tail|domestic|catsize|type|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----------+----+--------+-------+----+
|   aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|         4|   0|       0|      1|   1|
|   antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|         4|   1|       0|      1|   1|
|       bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|         0|   1|       0|      0|   4|
|       bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|         4|   0|       0|      1|   1|
|     

In [32]:
df = df.withColumn("new_hair", df.hair + 2)
df.show()

+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----------+----+--------+-------+----+--------+
|animal name|hair|feathers|eggs|milk|airborne|aquatic|predator|toothed|backbone|breathes|venomous|fins|no_of_legs|tail|domestic|catsize|type|new_hair|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----------+----+--------+-------+----+--------+
|   aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|         4|   0|       0|      1|   1|       3|
|   antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|         4|   1|       0|      1|   1|       3|
|       bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|         0|   1|       0|      0|   4|       2|
|       bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|     

In [33]:
df = df.drop("new_hair")
df.show()

+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----------+----+--------+-------+----+
|animal name|hair|feathers|eggs|milk|airborne|aquatic|predator|toothed|backbone|breathes|venomous|fins|no_of_legs|tail|domestic|catsize|type|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----------+----+--------+-------+----+
|   aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|         4|   0|       0|      1|   1|
|   antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|         4|   1|       0|      1|   1|
|       bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|         0|   1|       0|      0|   4|
|       bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|         4|   0|       0|      1|   1|
|     

In [68]:
df2 = spark.read.csv("/content/drive/MyDrive/dataset/netflix_titles.csv", header=True,inferSchema=True)
df2.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                NULL|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                NULL|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglan

In [69]:
# dropping multiple columns
df2 = df2.drop(*['cast', 'listed_in','description'])
df2.show()

+-------+-------+--------------------+--------------------+--------------------+------------------+------------+------+---------+
|show_id|   type|               title|            director|             country|        date_added|release_year|rating| duration|
+-------+-------+--------------------+--------------------+--------------------+------------------+------------+------+---------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|       United States|September 25, 2021|        2020| PG-13|   90 min|
|     s2|TV Show|       Blood & Water|                NULL|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|
|     s3|TV Show|           Ganglands|     Julien Leclercq|                NULL|September 24, 2021|        2021| TV-MA| 1 Season|
|     s4|TV Show|Jailbirds New Orl...|                NULL|                NULL|September 24, 2021|        2021| TV-MA| 1 Season|
|     s5|TV Show|        Kota Factory|                NULL|               India|September 

In [39]:
df2 = df2.dropna()
df2 = df2.na.drop(how ="all") # if a row contain all null values then only drops
df2 = df2.na.drop(how ="any") # if a row contains any single null then drops

df2.na.drop(thresh = 2) # keeps the row only if it has 2 non null values
df2.na.drop(subset = ['director']) # keeps the row only if director is not null
df2.show()

+-------+-------+--------------------+-------------------+--------------------+------------------+------------+------+---------+--------------------+
|show_id|   type|               title|           director|             country|        date_added|release_year|rating| duration|         description|
+-------+-------+--------------------+-------------------+--------------------+------------------+------------+------+---------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|    Kirsten Johnson|       United States|September 25, 2021|        2020| PG-13|   90 min|As her father nea...|
|     s8|  Movie|             Sankofa|       Haile Gerima|United States, Gh...|September 24, 2021|        1993| TV-MA|  125 min|On a photo shoot ...|
|     s9|TV Show|The Great British...|    Andy Devonshire|      United Kingdom|September 24, 2021|        2021| TV-14|9 Seasons|A talented batch ...|
|    s10|  Movie|        The Starling|     Theodore Melfi|       United States|September 24, 2021|  

In [45]:
df2.na.drop(subset = ['director']).show() # keeps the row only if director is not null
df2=df2.dropDuplicates()
df2.show()

+-------+-------+--------------------+--------------------+--------------------+------------------+------------+------+---------+
|show_id|   type|               title|            director|             country|        date_added|release_year|rating| duration|
+-------+-------+--------------------+--------------------+--------------------+------------------+------------+------+---------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|       United States|September 25, 2021|        2020| PG-13|   90 min|
|     s3|TV Show|           Ganglands|     Julien Leclercq|                NULL|September 24, 2021|        2021| TV-MA| 1 Season|
|     s6|TV Show|       Midnight Mass|       Mike Flanagan|                NULL|September 24, 2021|        2021| TV-MA| 1 Season|
|     s7|  Movie|My Little Pony: A...|Robert Cullen, Jo...|                NULL|September 24, 2021|        2021|    PG|   91 min|
|     s8|  Movie|             Sankofa|        Haile Gerima|United States, Gh...|September 

In [70]:
df2 = df2.na.fill("Missing", 'director') #filling single column
df2 = df2.na.fill("Missing", ['country','director']) # filling multiple column

In [71]:
df2 = df2.withColumn('dummy_col',  lit(10))

In [73]:
imputer = Imputer(inputCols=["dummy_col"], outputCols=["imputed_dummy_col"]).setStrategy("median")
model = imputer.fit(df2)
df2 = model.transform(df2)
df2.show()

+-------+-------+--------------------+--------------------+--------------------+------------------+------------+------+---------+---------+-----------------+
|show_id|   type|               title|            director|             country|        date_added|release_year|rating| duration|dummy_col|imputed_dummy_col|
+-------+-------+--------------------+--------------------+--------------------+------------------+------------+------+---------+---------+-----------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|       United States|September 25, 2021|        2020| PG-13|   90 min|       10|               10|
|     s2|TV Show|       Blood & Water|             Missing|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|       10|               10|
|     s3|TV Show|           Ganglands|     Julien Leclercq|             Missing|September 24, 2021|        2021| TV-MA| 1 Season|       10|               10|
|     s4|TV Show|Jailbirds New Orl...|             M

In [78]:
#filters
df2 = df2.filter(df2.release_year > 2000)
df2 = df2.filter(df2.type == "Movie").filter(df2.country == "India")
df2.show()

+-------+-----+--------------------+--------------------+-------+-----------------+------------+------+--------+---------+-----------------+
|show_id| type|               title|            director|country|       date_added|release_year|rating|duration|dummy_col|imputed_dummy_col|
+-------+-----+--------------------+--------------------+-------+-----------------+------------+------+--------+---------+-----------------+
|   s106|Movie|    Angamaly Diaries|Lijo Jose Pellissery|  India|September 5, 2021|        2017| TV-14| 128 min|       10|               10|
|   s117|Movie|              Dhanak|     Nagesh Kukunoor|  India|September 2, 2021|        2015| TV-PG| 114 min|       10|               10|
|   s119|Movie|             Gurgaon|       Shanker Raman|  India|September 2, 2021|        2017| TV-14| 106 min|       10|               10|
|   s127|Movie|             Shikara|  Vidhu Vinod Chopra|  India|September 2, 2021|        2020| TV-14| 115 min|       10|               10|
|   s191|Movi

In [82]:
df2.filter(df2.duration == "128 min").show()

+-------+-----+--------------------+--------------------+-------+------------------+------------+------+--------+---------+-----------------+
|show_id| type|               title|            director|country|        date_added|release_year|rating|duration|dummy_col|imputed_dummy_col|
+-------+-----+--------------------+--------------------+-------+------------------+------------+------+--------+---------+-----------------+
|   s106|Movie|    Angamaly Diaries|Lijo Jose Pellissery|  India| September 5, 2021|        2017| TV-14| 128 min|       10|               10|
|   s196|Movie|EMI: Liya Hai To ...|       Saurabh Kabra|  India|   August 27, 2021|        2008| TV-14| 128 min|       10|               10|
|   s542|Movie|    Truckbhar Swapna|        Pramod Pawar|  India|      July 6, 2021|        2018| TV-14| 128 min|       10|               10|
|   s960|Movie|        The Disciple|   Chaitanya Tamhane|  India|    April 30, 2021|        2021| TV-MA| 128 min|       10|               10|
|  s16

In [89]:
df2.filter(~(df2.release_year < 2000)).show()


+-------+-----+--------------------+--------------------+-------+-----------------+------------+------+--------+---------+-----------------+
|show_id| type|               title|            director|country|       date_added|release_year|rating|duration|dummy_col|imputed_dummy_col|
+-------+-----+--------------------+--------------------+-------+-----------------+------------+------+--------+---------+-----------------+
|   s106|Movie|    Angamaly Diaries|Lijo Jose Pellissery|  India|September 5, 2021|        2017| TV-14| 128 min|       10|               10|
|   s117|Movie|              Dhanak|     Nagesh Kukunoor|  India|September 2, 2021|        2015| TV-PG| 114 min|       10|               10|
|   s119|Movie|             Gurgaon|       Shanker Raman|  India|September 2, 2021|        2017| TV-14| 106 min|       10|               10|
|   s127|Movie|             Shikara|  Vidhu Vinod Chopra|  India|September 2, 2021|        2020| TV-14| 115 min|       10|               10|
|   s191|Movi

In [90]:
data = [
    ("John", "HR", 3000),
    ("Anna", "IT", 4000),
    ("Mike", "Finance", 5000),
    ("Anna", "Finance", 4000),
    ("Mike", "HR", 5000),
    ("Sara", "HR", 3500),
    ("Tom", "IT", 4500)
]
columns = ["Name", "Department", "Salary"]

df3 = spark.createDataFrame(data, columns)

# Show the DataFrame
df3.show()

+----+----------+------+
|Name|Department|Salary|
+----+----------+------+
|John|        HR|  3000|
|Anna|        IT|  4000|
|Mike|   Finance|  5000|
|Anna|   Finance|  4000|
|Mike|        HR|  5000|
|Sara|        HR|  3500|
| Tom|        IT|  4500|
+----+----------+------+



In [95]:
df3.groupBy("Name").sum().show()
df3.groupBy("Department").sum().show()
df3.groupBy("Department").mean().show()
df3.groupBy("Department").count().show()

+----+-----------+
|Name|sum(Salary)|
+----+-----------+
|Anna|       8000|
|John|       3000|
|Mike|      10000|
| Tom|       4500|
|Sara|       3500|
+----+-----------+

+----------+-----------+
|Department|sum(Salary)|
+----------+-----------+
|        HR|      11500|
|   Finance|       9000|
|        IT|       8500|
+----------+-----------+

+----------+------------------+
|Department|       avg(Salary)|
+----------+------------------+
|        HR|3833.3333333333335|
|   Finance|            4500.0|
|        IT|            4250.0|
+----------+------------------+

+----------+-----+
|Department|count|
+----------+-----+
|        HR|    3|
|   Finance|    2|
|        IT|    2|
+----------+-----+

